# Christmas Tree Lights - Various Effects
171209<br>
171210<br>
191215 Using for 2019 Xmas - simple string of 4 x 300. Mapped physical positions <br> 
201205 2020 Xmas

Note that you need to edit the actual librray to change the LED numbers and boundaries <br>
See /home/pi/DotStar/luc/python-dotstar-lights/dotstar/tree_lights.py <br>

2020
I should look at upgrading to Python 3 
Micropython version is here - but is it compatible with Python 3?
https://learn.adafruit.com/circuitpython-essentials/circuitpython-dotstar

Can I sync to lights?



## Setup

In [1]:
import sys

# Have to remove this version of path from the system path, otherwise can load tree_lights
# sys.path.remove('/home/pi/DotStar/luc/python-dotstar-lights/dotstar')

libpath = "/home/pi/DotStar/luc/python-dotstar-lights"
if not( libpath in sys.path):
    sys.path.append(libpath)

from dotstar import tree_lights
#import dotstar

import time
import random
import numpy as np
import math

global flagZero
flagZero = False 
# Use to see if lights have been already been zeroed, to avoid doing it repeatedly

In [2]:
# Matplotlib provides RGB to HSV conversion function: matplotlib.colors.rgb_to_hsv():
# matplotlib.colors.rgb_to_hsv(arr)
# convert rgb values in a numpy array to hsv values input and output arrays should have shape (M,N,3)

import matplotlib.pyplot as plt
import matplotlib as mpl


### Time keeping

In [19]:
def waitHereDuringSleepPeriods():
    global flagZero
    
    flag_NeverSleep = False   # True - Will stop any sleep logic, use while testing
    flag_forceWeekend = True
    
    # flagZero is used so we don't keep sending tree.zeroAllPixels()
    tt = time.time()
    h = (time.gmtime(tt).tm_hour - 5)%24 # Hour of day EST
    m = time.gmtime(tt).tm_min
    s = time.gmtime(tt).tm_sec
    w = time.gmtime(tt).tm_wday # 0 is MONDAY
    
    if flag_forceWeekend: # SET ALWAYS TO WEEKEND SCHEDULE
        w = 6

    # Can comment this out
    # print("Day %1d  Time %02d:%02d:%02d"%(w,h,m,s ))

    if m==0 and s==0:
        print("Day %1d  Time %02d:%02d:%02d"%(w,h,m,s ))
        time.sleep(1.)
    if ( not(flag_NeverSleep) # flag_NeverSleep = True - stops it from sleeping
        and
        (
        ( 
            (w>=0 and w<=4) # Weekday
            and 
            (  
            (h>=9 and h<=15)# 0900-1500 off
            or(h>=0 and h<=6)# 0000-0500 off
            )
        ) 
        or
        (
            (w>=5 and w<=6) # Weekend
            and
            (
            (h>=0 and h<=8) # 0000-0800 off
            )
        )
        )
    ):
        if not flagZero: #
            tree.zeroAllPixels()
            flagZero = True    # Remember we have already zeroed lights
            print( "Tree is sleeping (hour=%d) - will start again in while"%h )
            time.sleep( 60*60 ) # Sleep for 1 hour
        else:
            flagZero = False # we have lights on

### Color Temperature RGB

In [4]:
kelvin_table = {
    1000: (255, 56, 0),
    1100: (255, 71, 0),
    1200: (255, 83, 0),
    1300: (255, 93, 0),
    1400: (255, 101, 0),
    1500: (255, 109, 0),
    1600: (255, 115, 0),
    1700: (255, 121, 0),
    1800: (255, 126, 0),
    1900: (255, 131, 0),
    2000: (255, 138, 18),
    2100: (255, 142, 33),
    2200: (255, 147, 44),
    2300: (255, 152, 54),
    2400: (255, 157, 63),
    2500: (255, 161, 72),
    2600: (255, 165, 79),
    2700: (255, 169, 87),
    2800: (255, 173, 94),
    2900: (255, 177, 101),
    3000: (255, 180, 107),
    3100: (255, 184, 114),
    3200: (255, 187, 120),
    3300: (255, 190, 126),
    3400: (255, 193, 132),
    3500: (255, 196, 137),
    3600: (255, 199, 143),
    3700: (255, 201, 148),
    3800: (255, 204, 153),
    3900: (255, 206, 159),
    4000: (255, 209, 163),
    4100: (255, 211, 168),
    4200: (255, 213, 173),
    4300: (255, 215, 177),
    4400: (255, 217, 182),
    4500: (255, 219, 186),
    4600: (255, 221, 190),
    4700: (255, 223, 194),
    4800: (255, 225, 198),
    4900: (255, 227, 202),
    5000: (255, 228, 206),
    5100: (255, 230, 210),
    5200: (255, 232, 213),
    5300: (255, 233, 217),
    5400: (255, 235, 220),
    5500: (255, 236, 224),
    5600: (255, 238, 227),
    5700: (255, 239, 230),
    5800: (255, 240, 233),
    5900: (255, 242, 236),
    6000: (255, 243, 239),
    6100: (255, 244, 242),
    6200: (255, 245, 245),
    6300: (255, 246, 247),
    6400: (255, 248, 251),
    6500: (255, 249, 253),
    6600: (254, 249, 255),
    6700: (252, 247, 255),
    6800: (249, 246, 255),
    6900: (247, 245, 255),
    7000: (245, 243, 255),
    7100: (243, 242, 255),
    7200: (240, 241, 255),
    7300: (239, 240, 255),
    7400: (237, 239, 255),
    7500: (235, 238, 255),
    7600: (233, 237, 255),
    7700: (231, 236, 255),
    7800: (230, 235, 255),
    7900: (228, 234, 255),
    8000: (227, 233, 255),
    8100: (225, 232, 255),
    8200: (224, 231, 255),
    8300: (222, 230, 255),
    8400: (221, 230, 255),
    8500: (220, 229, 255),
    8600: (218, 229, 255),
    8700: (217, 227, 255),
    8800: (216, 227, 255),
    8900: (215, 226, 255),
    9000: (214, 225, 255),
    9100: (212, 225, 255),
    9200: (211, 224, 255),
    9300: (210, 223, 255),
    9400: (209, 223, 255),
    9500: (208, 222, 255),
    9600: (207, 221, 255),
    9700: (207, 221, 255),
    9800: (206, 220, 255),
    9900: (205, 220, 255),
    10000: (207, 218, 255),
    10100: (207, 218, 255),
    10200: (206, 217, 255),
    10300: (205, 217, 255),
    10400: (204, 216, 255),
    10500: (204, 216, 255),
    10600: (203, 215, 255),
    10700: (202, 215, 255),
    10800: (202, 214, 255),
    10900: (201, 214, 255),
    11000: (200, 213, 255),
    11100: (200, 213, 255),
    11200: (199, 212, 255),
    11300: (198, 212, 255),
    11400: (198, 212, 255),
    11500: (197, 211, 255),
    11600: (197, 211, 255),
    11700: (197, 210, 255),
    11800: (196, 210, 255),
    11900: (195, 210, 255),
    12000: (195, 209, 255)}

In [5]:
kelvin_table[2300]

(255, 152, 54)

### LED Manipulation Version 1

In [4]:
def twinkleArray(ia):
    a = ia.astype(float)
    # Twinkle values in array using adjustment that is based on the value of pixel
    # Amplitude of twinkle is  pixel +/- sqrt(pixel) [rand between limits]
    a = (a 
           + ( np.sqrt(abs(a)) * (2.* (np.random.rand(a.size)-0.5) ) )
        ) 
    return( a.astype(int) )


def writeArrayIndexList(indexList , amplitude , col_1 , min_1=0 ): 
    # indexList are LED locations
    # Amplitude can be same length floats that multiplies col g r b vector
    # Array a (amplitude) * col (3 element) + min (1 or 3 element)
    # All are numpy arrays
    
    # If amplitude is slingle value, then make it an array
    if not( hasattr(amplitude, "__len__") ):
        amplitude = amplitude * np.full_like(indexList, 1.)
    for (n,i) in enumerate(indexList):
        tree.setPixel( i , *( ( min_1+(amplitude[n] * col_1) ).astype(int) ) )


def writeArray(a,col_1,min_1=0): 
    # Array a (amplitude) * col (3 element) + min (1 or 3 element)
    # All are numpy arrays
    for i in range(a.size):
        tree.setPixel( i , *( ( min_1+(a[i] * col_1) ).astype(int) ) )

def writeArray2(a ,col_1 ,col_2 ,min_1=0 ,min_2=0 ): 
    # Array a (amplitude) * col (3 element) + min (1 or 3 element)
    # All are numpy arrays
    for i in range(a.size):
        if a[i]>0:
            tree.setPixel( i , *( ( min_1+(a[i] * col_1) ).astype(int) ) )
        else:
            tree.setPixel( i , *( ( min_2+(abs(a[i]) * col_2) ).astype(int) ) )
            
            
def writeHSVIndexList(indexList , hue , saturation , brightness ): # NOT DONE YET
    # indexList are LED locations
    # Amplitude can be same length floats that multiplies col g r b vector
    # Array a (amplitude) * col (3 element) + min (1 or 3 element)
    # All are numpy arrays
    
    # If amplitude is slingle value, then make it an array
    if not( hasattr(amplitude, "__len__") ):
        amplitude = amplitude * np.full_like(indexList, 1.)
    for (n,i) in enumerate(indexList):
        tree.setPixel( i , *( ( min_1+(amplitude[n] * col_1) ).astype(int) ) )



## Setup Regions - Pixel Start and End

( This is from "Find the position of LEDs.ipynb" )<br>
191212 RG Xmas 4 x 300 strips only <br>
Couldn't get anymore lights to work, even with slower clock speed <br>
<br>

171210 RG Xmas light Positions <br>
Total Pixels:            (1104)<br>  
Strip:            0- 592 ( 593)<br>
Disk:           593- 847 ( 255)<br>
Square:         848-1103 ( 256)<br>
Starts at Bottom Outer, going anticlockwise <br>
With wiring at top, starts top left, going downwards, then snakes up, ends top right


#### Single Long Strip

In [5]:
# This is from "Find the position of LEDs.ipynb"

# NUM_PIXELS = 1104  # 1 Strip                                                                               
# See end for explicit calculation of NUM_PIXELS

if 1:
    # All positions assume first pixel is ZERO
    # Strip
    lengthOfStrip = (4*300-12)
    startOfStrip = 0
    endOfStrip = startOfStrip + (lengthOfStrip - 1)
    # Base:                                                                                                                                  
    startOfBase = startOfStrip
    endOfBase = endOfStrip

    # Disk:                                                                                                                                  
    lengthOfDisk = 0
    startOfDisk = endOfStrip+1
    endOfDisk = startOfDisk + (lengthOfDisk-1)

    # Square:                                                                                                                                  
    lengthOfSquare = 0
    startOfSquare = endOfDisk + 1
    endOfSquare = startOfSquare + (lengthOfSquare-1)

NUM_PIXELS = lengthOfStrip + lengthOfDisk + lengthOfSquare  # 3 Strips + Disk + Square                                                                               


print( "Total Pixels:            (%4d)      " % NUM_PIXELS )
print( "Strip:         %4d-%4d (%4d)"% (startOfStrip,endOfStrip,lengthOfStrip))
print( "Disk:          %4d-%4d (%4d)"% (startOfDisk,endOfDisk,lengthOfDisk))
print( "Square:        %4d-%4d (%4d)"% (startOfSquare,endOfSquare,lengthOfSquare))

Total Pixels:            (1188)      
Strip:            0-1187 (1188)
Disk:          1188-1187 (   0)
Square:        1188-1187 (   0)


#### Full Setup 

In [ ]:
# This is from "Find the position of LEDs.ipynb"

# NUM_PIXELS = 1104  # 2 Strips + Disk + Square                                                                               
# See end for explicit calculation of NUM_PIXELS

if 1:
    # All positions assume first pixel is ZERO
    # Strip
    lengthOfStrip = (4*300-7)
    startOfStrip = 0
    endOfStrip = startOfStrip + (lengthOfStrip - 1)
    # Base:                                                                                                                                  
    startOfBase = startOfStrip
    endOfBase = endOfStrip

    # Disk:                                                                                                                                  
    lengthOfDisk = 0
    startOfDisk = endOfStrip+1
    endOfDisk = startOfDisk + (lengthOfDisk-1)

    # Square:                                                                                                                                  
    lengthOfSquare = 0
    startOfSquare = endOfDisk + 1
    endOfSquare = startOfSquare + (lengthOfSquare-1)

NUM_PIXELS = lengthOfStrip + lengthOfDisk + lengthOfSquare  # 3 Strips + Disk + Square                                                                               


print( "Total Pixels:            (%4d)      " % NUM_PIXELS )
print( "Strip:         %4d-%4d (%4d)"% (startOfStrip,endOfStrip,lengthOfStrip))
print( "Disk:          %4d-%4d (%4d)"% (startOfDisk,endOfDisk,lengthOfDisk))
print( "Square:        %4d-%4d (%4d)"% (startOfSquare,endOfSquare,lengthOfSquare))

## Turn All Lights Off

In [6]:
# TURN ALL LIGHTS OFF
tree = tree_lights.TreeLights()  # slower rate
tree.zeroAllPixels()


## Set one pixel after another

In [6]:
j = 100

In [7]:
# Put single red pixel 

updateDelay = 0.001 # secs

try:
    while j < range(NUM_PIXELS):
        j=j+1
        tree.zeroAllPixels()
        flagZero = True    # Remember we have already zeroed lights

        stringCenters = np.array([j,])


        for i in stringCenters:
            color = (0,64,0)
            tree.setPixel( i , *color)

        tree.show()                     # Refresh strip
        flagZero = False    # Remember we have set lights

except KeyboardInterrupt:
        # Shutdown pixels on interrupt:
        tree.zeroAllPixels()
        flagZero = True    # Remember we have already zeroed lights

print( j )


#mapFromPositionToLEDNumberFunc = np.poly1d( np.polyfit( stringIndex , stringCenters , deg = 3) )
#mapFromLEDNumberToPositionFunc = np.poly1d( np.polyfit( stringCenters , stringIndex ,  deg = 4) )
#mapFromPositionToLEDNumberFunc

224


## Simple Chase Effect

In [10]:
## Simple Chase Effect

# Initialize the strip:
tree = tree_lights.TreeLights()  # slower rate
flagZero = False

# Now set above
# DISK SWIRL:
#startOfDisk = tree_lights.TreeLightSectionPositions.startOfDisk
#endOfDisk = tree_lights.TreeLightSectionPositions.endOfDisk

swirlHead  = startOfDisk    # Index of first 'on' pixel
swirlTail  = startOfDisk-10  # Index of last 'off' pixel
swirlColor = [64, 0, 0]

swirlEnd = endOfSquare

def swirlDiskStep():
    """Step the swirl on the disk at the top of the tree."""
    global swirlHead
    global swirlTail
    global swirlColor

    tree.setPixel(swirlHead, *swirlColor)  # Turn on 'head' pixel
    tree.setPixel(swirlTail, 0)     # Turn off 'tail'

    swirlHead += 1                        # Advance head position
    if(swirlHead >= swirlEnd):           # Off end of strip?
        swirlHead = startOfDisk              # Reset to start

    swirlTail += 1                        # Advance tail position
    if(swirlTail >= swirlEnd):
        swirlTail = startOfDisk  # Off end? Reset

    appendColor = swirlColor[0]
    del swirlColor[0]
    swirlColor.append(appendColor)


# TWINKLE TREE BASE:
startOfBase = tree_lights.TreeLightSectionPositions.startOfBase
endOfBase = tree_lights.TreeLightSectionPositions.endOfBase
# print('Start of base %d, End of Base %d\n'%(startOfBase , endOfBase))

colorChangeIndex = 0

def baseTwinkleOnce():
    """Twinkle the base lights on the tree once."""
    global colorChangeIndex

    for px in range(startOfBase, endOfBase):
        # Get a random number in a specific range:
        # randomNumb = random.randrange(4, 23, 1)
        randomNumb = random.randrange(4, 2*23, 1)
        color = []

        for ii in range(colorChangeIndex):
            color.append(0)
        color.append(randomNumb)
        for ii in range(3 - (colorChangeIndex + 1)):
            color.append(0)

        colorChangeIndex += 1
        if colorChangeIndex > 2:
            colorChangeIndex = 0
        tree.setPixel(px, *color)


if __name__ == "__main__":
    
    
    try:
        swirlTime = time.time()
        swirlSleepTime = 1000000 # large if not using, 0.001 if are using
        twinkleTime = time.time()
        twinkleSleepTime = 0.33       
        
        while True:

            waitHereDuringSleepPeriods() # Waits in this function when don't want flash
            
            # Test to see if nightime , adjust from GMT to EST
            if time.time() - swirlTime > swirlSleepTime:
                # Update the Disk Swirl:
                swirlDiskStep()
                swirlTime = time.time()
                tree.show()                     # Refresh strip
                flagZero = False    # Remember we have set lights

            if time.time() - twinkleTime > twinkleSleepTime:
                # Update base twinkle:
                baseTwinkleOnce()
                twinkleTime = time.time()
                tree.show()                     # Refresh strip
                flagZero = False    # Remember we have set lights
                    
    except KeyboardInterrupt:
        # Shutdown pixels on interrupt:
        tree.zeroAllPixels()
        flagZero = True    # Remember we have already zeroed lights




## Red-Green Oscillating Twinkle 

In [ ]:
beatPeriod = 120.   # secs for full cycle
updateDelay = 0.01 # Update delay in secs

# Initialize the strip:
tree = tree_lights.TreeLights()  


col_1 = np.array([128.,0,0]) # Green
col_2 = np.array([0,128.,0]) # Red
b_2 = 4. * np.array([1.,0,0])
b_1 = 4. * np.array([0,1.,0])
x = np.ones(NUM_PIXELS)

if __name__ == "__main__":
    try:
        t0 = time.time()
        
        while True:
            waitHereDuringSleepPeriods() # Waits in this function when don't want flash

            t = time.time()
            a = np.sin(2.*math.pi*t/beatPeriod)
            # print( "%7.3f %7.3f"%(t-t0,a) )
            writeArray2( twinkleArray(a*x) ,col_1 ,col_2 ,b_1 ,b_2 )
                
            tree.show()                     # Refresh strip
            time.sleep(updateDelay)
            
    except KeyboardInterrupt:
        # Shutdown pixels on interrupt:
        tree.zeroAllPixels()



Tree is sleeping (hour=1) - will start again in while


In [ ]:
col_1 = np.array([64,0,0])
col_2 = np.array([0,64,0])

4*col_1

In [9]:
tree.zeroAllPixels()

### White Twinkle

In [ ]:
beatPeriod = 120.   # secs for full cycle
updateDelay = 0.01 # Update delay in secs

# Initialize the strip:
tree = tree_lights.TreeLights()  


col_1 = 64. * np.array([1.,1.,1.]) # Green
col_2 = 16. * np.array([1.,1.,1.]) # Red
b_2 = 0. * np.array([1.,1.,1.])
b_1 = 0. * np.array([1.,1.,1.])
x = np.ones(NUM_PIXELS)

if __name__ == "__main__":
    try:
        t0 = time.time()
        
        while True:
            waitHereDuringSleepPeriods() # Waits in this function when don't want flash

            t = time.time()
            a = np.sin(2.*math.pi*t/beatPeriod)
            # print( "%7.3f %7.3f"%(t-t0,a) )
            writeArray2( twinkleArray(a*x) ,col_1 ,col_2 ,b_1 ,b_2 )
                
            tree.show()                     # Refresh strip
            time.sleep(updateDelay)
            
    except KeyboardInterrupt:
        # Shutdown pixels on interrupt:
        tree.zeroAllPixels()




Day 4  Time 17:00:00


## Sectors (this block is not finished - see further down)

Lights go round tree 7 times. 
The circumference gets smaller as it goes to higher indices.
Attempt to model this 

In [ ]:
colorChangeIndex = 0 # Keep track of colors used globally

sector_num = 21
sector_shorter_fraction = -0.05 # fraction gets shorter for each new sector

# Calculate the sector lengths that add to the total length 
sector_length = ( 1 + sector_shorter_fraction * np.arange( sector_num ) )
sector_length = ( np.floor( sector_length * float(NUM_PIXELS) / np.sum( sector_length ) ) )
# print( np.sum( sector_length ) , sector_length )
sector_boundaries = [ int(ii) for ii in np.append( np.array([0]) , np.cumsum( sector_length )  ) ]

In [ ]:
# Put single red pixel on center line

tree.zeroAllPixels()
flagZero = True    # Remember we have already zeroed lights

for s in np.arange( -0.5 , 7.5, 0.5 ):
    color = (0,64,0)
    tree.setPixel( mapFromPositionToLEDNumber(s) , *color)

tree.show()                     # Refresh strip
flagZero = False    # Remember we have set lights

In [ ]:
def simpleSectors():
    global colorChangeIndex


In [ ]:
colorlist = ( [ [0,64,0], [64,0,0] , [0,0,64] ] )  # [ g , r , b] 
colorindex = 0

for s in range( len(sector_boundaries)-1 ):
    # print( 'Sector Index %d covering %d - %d ' % ( s ,  sector_boundaries[s] , sector_boundaries[s+1] ) )
    color = colorlist[ colorindex ]
    # print( '   ' , colorindex , color )
    for px in range( sector_boundaries[s] , sector_boundaries[s+1] ):         
        tree.setPixel(px, *color)
    if colorindex < len( colorlist )-1:
        colorindex = colorindex + 1 
    else:
        colorindex = 0 
tree.show()                     # Refresh strip
flagZero = False    # Remember we have set lights


In [ ]:
try:
    lastTime = time.time()
    sleepTime = 0.01 # large if not using, 0.001 if are using

    while True:

        waitHereDuringSleepPeriods() # Waits in this function when don't want flash
        # Test to see if nightime , adjust from GMT to EST

        if time.time() - lastTime > sleepTime:
            # Show sectors
            simpleSectors()
            lastTime = time.time()
            tree.show()                     # Refresh strip
            flagZero = False    # Remember we have set lights

except KeyboardInterrupt:
    # Shutdown pixels on interrupt:
    tree.zeroAllPixels()
    flagZero = True    # Remember we have already zeroed lights




## Hand Calibrate Physical positions of pixels on the tree - use a center line at front

In [8]:
# Set and show the center line

# Position on LED string of the Center Line at front - measured manually
# First and last are fake just to help interpolation

stringCenters = np.array(
    [110+4-(258-110) # Fake point for interpolation
    ,110+4
    ,285+2
    ,445
    ,604-1
    ,728+2
    ,857
    ,978
    ,1090
    ,1165
    ,1165+(1165-1090) # Fake point for interpolation
    ]
)
stringIndex = np.arange(len(stringCenters))

# Calculate backs that divide the LEDs into rings
stringBacks = np.int32( (stringCenters[:-1]+stringCenters[1:])/2. )
stringBacks = np.maximum( stringBacks , 0)
stringBacks = np.minimum( stringBacks , NUM_PIXELS)
if stringBacks[0] > 0:
    stringBacks = np.concatenate(( (0,),stringBacks ) )
if stringBacks[-1] < NUM_PIXELS:
    stringBacks = np.concatenate(( stringBacks,(NUM_PIXELS,) ) )

In [9]:
# Put single red pixel on center line

tree.zeroAllPixels()
flagZero = True    # Remember we have already zeroed lights

for i in stringCenters:
    color = (0,64,0)
    tree.setPixel( i , *color)

tree.show()                     # Refresh strip
flagZero = False    # Remember we have set lights


mapFromPositionToLEDNumberFunc = np.poly1d( np.polyfit( stringIndex , stringCenters , deg = 3) )
mapFromLEDNumberToPositionFunc = np.poly1d( np.polyfit( stringCenters , stringIndex ,  deg = 4) )

mapFromPositionToLEDNumberFunc

poly1d([  -0.30439005,   -0.78496503,  166.12432012,  -39.8951049 ])

In [10]:
mapFromLEDNumberToPositionFunc

poly1d([  1.90703367e-12,  -1.99765278e-09,   9.79754791e-07,
         6.06979342e-03,   2.32507347e-01])

In [11]:
# Linear space is -0.5 to 7.5 with tree center line on integers
# The LED number space is provided by mapFromPositionToLEDNumber()
#
# Fit baased on polynomial - doesn't quiute get the center line - due to smoothing - better to use crude interp
def mapFromPositionToLEDNumber( s ):
    return( int( np.minimum( 1200. , np.maximum(0. , 
        mapFromPositionToLEDNumberFunc(s)           
    ) ) ) )

def mapFromLEDNumberToPosition( i ):
    return( mapFromLEDNumberToPositionFunc(i) )

In [12]:
# Try linear interpolations - this gets the center line better

def mapFromPositionToLEDNumber( s ):
    return( int( np.minimum( 1200. , np.maximum(0. , 
        np.interp(s,stringIndex,stringCenters)           
    ) ) ) )

def mapFromLEDNumberToPosition( i ):
    return( np.interp(i,stringCenters,stringIndex) )

In [13]:
mapFromPositionToLEDNumber( 2. )

287

In [14]:
# Put single red pixel on center line based on the interpolation
# To check 

tree.zeroAllPixels()
flagZero = True    # Remember we have already zeroed lights

#for s in np.arange( -0.5 , 7.5, 0.5 ):
for s in np.arange( 0. , 7.1, 1.0 ):
    color = (0,64,0)
    tree.setPixel( mapFromPositionToLEDNumber(s) , *color)

tree.show()                     # Refresh strip
flagZero = False    # Remember we have set lights

## Sectors that run down the tree vertically - based on a hand calibrated center line - this works!!! 2019

In [ ]:
# Find the linear space values for each LED index
pos = mapFromLEDNumberToPosition( np.arange(NUM_PIXELS) )

beatPeriod = 4 # 2 , 0.8   # secs for full cycle
updateDelay = 0.01 # Update delay in secs

beatWavelength = 0.2 # 0.33 # 1. is once round tree

# Initialize the strip:
tree = tree_lights.TreeLights()  

tree.zeroAllPixels()
    
col_1 = np.array([16.,16.,16.]) # White
b_1 = 4. * np.array([1.,1.,1.])

col_1 = np.array([64.,0.,0.]) # Green
b_1 = 4. * np.array([1.,0.,0.])

col_2 = np.array([0,64.,0]) # Red
b_2 = 4. * np.array([0,1.,0])

x = np.ones(NUM_PIXELS)

try:
    t0 = time.time()

    while True:
        waitHereDuringSleepPeriods() # Waits in this function when don't want flash

        t = time.time()
        a = np.sin(2.*math.pi*(pos/beatWavelength - (t-t0)/beatPeriod))
        
        # print( "%7.3f %7.3f"%(t-t0,a) )
        writeArray2( a*x ,col_1 ,col_2 ,b_1 ,b_2 )

        tree.show()                     # Refresh strip
        time.sleep(updateDelay)

except KeyboardInterrupt:
    # Shutdown pixels on interrupt:
    tree.zeroAllPixels()

Day 0  Time 20:00:00


## Horizontal rings - based on a hand calibrated sectors - this works!!! 2020

In [34]:
rgb = np.array( kelvin_table[2300] )/ 256.
print( rgb[ [1,0,2] ] ) # grb

[ 0.99609375  0.59375     0.2109375 ]
[ 0.59375     0.99609375  0.2109375 ]


In [ ]:
col_2

In [46]:
# Rings of different colors
# x = np.range(NUM_PIXELS)
updateDelay = 0.1 # sec

beatPeriod = 100. # sec

if 0:
    col_1 = np.array([64.,0.,0.]) # Green
    col_2 = np.array([0,16.,0]) # Red

if 0:
    col_1 = 8.  * np.array([1.,1.,1.]) # Green
    col_2 = 32. * np.array([1.,1.,1.]) # Red
    
if 1: # Color temperature
    grb = (np.array( kelvin_table[3000] )/ 256.)[ [1,0,2] ]
    col_1 = 64.  * grb # 
    col_2 = 128. * grb #

t0 = time.time()

flag = -1
while True:
    waitHereDuringSleepPeriods() # Waits in this function when don't want flash

    t = time.time()
    amplitude = 0.3 + 0.7*( np.sin(2.*math.pi*((t-t0)/beatPeriod)) )**2.
    # amplitude = 1

    for p in zip(stringBacks[:-1],stringBacks[1:]):
        indexList = np.arange(p[0],p[1])
        
        flag = flag+1
        if flag <= 1: #Most bands - Green
            writeArrayIndexList(indexList , amplitude , col_1 )
        else: # Single band - red
            writeArrayIndexList(indexList , amplitude , col_2 )
            flag = -1 # Go round again
    
    tree.show()                     # Refresh strip
    flagZero = False    # Remember we have set lights
    
    # flag = flag+1
    
    time.sleep(updateDelay)

Day 1  Time 22:00:00


KeyboardInterrupt: 

In [ ]:
1

## HSV

In [54]:
updateDelay = 1. # sec

indexList = np.arange( 710 , 721 )

In [55]:
# Set same amplitude for many and then increase in tie steps
for x in np.arange(0.,255.,16.):
    amplitude = np.full_like(indexList, x )

    col_1 = np.array([0.,1.,0.]) # Red

    for (n,i) in enumerate( indexList ):
        tree.setPixel( i , *( (amplitude[n] * col_1 ).astype(int) ) )
    tree.show()                     # Refresh strip
    time.sleep(updateDelay)
    
tree.zeroAllPixels()

In [50]:
tree.zeroAllPixels()

In [51]:

# Display a range of smplitudes

# Steps of 8 are resolvable, 4 much less so
# Above 128 is very similar - I should make an intensity scaling that is linear
#

tree.zeroAllPixels()

indexList = np.arange( 700 , 732 )

amplitude = np.full_like(indexList, x )

col_1 = np.array([0.,1.,0.]) # Red

for (n,i) in enumerate( indexList ):
    a = 32. * n
    print('Amp = %7.1f'%a )
    tree.setPixel( i , *( (a * col_1 ).astype(int) ) )
tree.show()                     # Refresh strip

Amp =     0.0
Amp =    32.0
Amp =    64.0
Amp =    96.0
Amp =   128.0
Amp =   160.0
Amp =   192.0
Amp =   224.0
Amp =   256.0


OverflowError: unsigned byte integer is greater than maximum

In [66]:
# Conversion
rgb = np.array( (1.,0.,0.) )
hsv = mpl.colors.rgb_to_hsv( rgb )
print( hsv , mpl.colors.hsv_to_rgb( hsv ))

(array([ 0.,  1.,  1.]), array([ 1.,  0.,  0.]))


In [69]:
mpl.colors.hsv_to_rgb( hsv )

array([ 1.,  0.,  0.])

In [81]:
rgb = np.ones((1,5)) * np.array( (1.,0.,0.) )

ValueError: operands could not be broadcast together with shapes (1,5) (3,) 

In [75]:
rgb

array([ 1.,  0.,  0.])

In [80]:
np.ones((1,5))

array([[ 1.,  1.,  1.,  1.,  1.]])

Need to add hsv array creation and handling to see if I can create waves acorss the tree
Will need to apply xy coord that is converted to LED index


# Random Inspired Effects

In [91]:
# Rings of different colors
# x = np.range(NUM_PIXELS)
updateDelay = 0.1 # sec

beatPeriod = 100. # sec

if 0:
    col_1 = np.array([64.,0.,0.]) # Green
    col_2 = np.array([0,16.,0]) # Red


t0 = time.time()

NP = NUM_PIXELS
# NP = 5
a = np.zeros( (NP,3) )
a[0] = np.random.random_sample((1,3)) # Some of algorithms use the first elememt as a seed

flag = -1
while True:
    waitHereDuringSleepPeriods() # Waits in this function when don't want flash

    t = time.time()
    # amplitude = 0.3 + 0.7*( np.sin(2.*math.pi*((t-t0)/beatPeriod)) )**2.
    # amplitude = 1

    skew = 5.
    renderMethod = 1    
    
    # Create a random grb values in range 0->1. and possibly sort them
    if 0: # Simple random colors with a bias for dominat g,r,b
        a = np.random.random_sample((NP,3)) # grb array 
        a = a[np.argsort(a[:, 0])] # Sort rows by first value
        a = a[np.argsort(a[:, 1])] # Sort rows by ? value
        a = a[np.argsort(a[:, 2])] # Sort rows by ? value
        skew = 5.
        renderMethod = 1
        
    # Create an initial color and then slowly morph it
    # If value overflows then wrap it?
    # 201207 Should also plot what it is doing to undestand functin better
    # I should do this in HSV space and H would wrap automatically - S and V should be pulled to a central value
    #

    if 1:
        seed = a[0]
        for i in np.arange(NP):
            seed = np.mod( seed + 0.01 * (np.random.random_sample(size=(1,3))-0.5) , 1.)
            a[i] = seed
        skew = 2.
        renderMethod = 1    



#    for i in np.arange(NUM_PIXELS):
#        if 0: # Simple random colors with a bias for dominat g,r,b
#            tree.setPixel( 
#                i , 
#                *( (128. * np.random.random_sample((3,1))**3. ).astype(int) ) # grb array expanded as 3 int arguments
#            )   
    
    # Write grb values from array a[i,3]
    for i in np.arange(NP):
        if renderMethod == 1:
            tree.setPixel( 
                i , 
                *( (128. * a[i]**skew ).astype(int) ) # grb array expanded as 3 int arguments
            )
            
        
    tree.show()                     # Refresh strip
    flagZero = False    # Remember we have set lights
    
    # flag = flag+1
    
    time.sleep(updateDelay)

Day 1  Time 23:00:00


KeyboardInterrupt: 

In [61]:
a = np.random.random_sample((NP,3))

In [74]:
np.mod( np.array( (1.4,0.3,2.3) ), 1. ) 

array([ 0.4,  0.3,  0.3])

In [77]:
seed

array([[ 0.57285594,  0.34728599,  0.76939891]])

In [83]:
a[0]

array([ 0.6755448 ,  0.04186577,  0.72947593])

In [82]:
a[0] = np.random.random_sample((1,3))

In [86]:
np.random.random_sample(size=(1,3))

array([[ 0.45807109,  0.54927408,  0.84094728]])